<a href="https://colab.research.google.com/github/BtC04ShW/Brobananaturnsred/blob/master/Bana_Invincible_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bana Invincible v2

### Mount Google Drive
To stream files we need to mount Google Drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

###Dependency
https://www.libtorrent.org/

In [0]:
!apt install python3-libtorrent

### Code to download torrent
Variable **link** stores the link string.

In [0]:
import libtorrent as lt
import time
import datetime
import sys

def getTracker():
    import urllib.request
    r = urllib.request.urlopen("https://btc04shw.github.io/tklist/tklist.txt")
    trackertBytes = r.read().decode("utf-8")
    s_ = "".join(trackertBytes.splitlines())
    s_ = s_.replace("http", "&tr=http").replace("udp:", "&tr=udp:")
    trackerStr = urllib.parse.quote(s_, safe='&=')
    return trackerStr

def convert_bytes(num):
    unit_ = 2**10
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < unit_:
            return f"{num:.2f} {x}"
        num /= unit_

ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': '/content/drive/My Drive/TorrentDLER/',
    'storage_mode': lt.storage_mode_t(2),
    'paused': False,
    'auto_managed': True,
    'duplicate_is_error': True}
link = "https://yts.lt/torrent/download/866BDCFA006930A718ADBC21D8CAE1F2C7F9D8B3" # PASTE TORRENT/MAGNET LINK HERE
print(link)
link += getTracker()

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

torinfo = handle.get_torrent_info()
print("Total size: {0} ({1})".format(convert_bytes(torinfo.total_size()), torinfo.total_size()))
print("Total numb:", torinfo.files().num_files())
#for index in range(torinfo.files().num_files()):
#    if (index >=0 and index <= 1):
#        handle.file_priority(index, 4)
#    else:
#        handle.file_priority(index, 0)
#    print("[" + str(index) + "]" + torinfo.files().file_path(index) + "|size:" + str(torinfo.files().file_size(index)) + "|priority:" +str(handle.file_priority(index)))

print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating', 'checking fastresume']
    sys.stdout.write('\r')
    sys.stdout.write('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    sys.stdout.flush()
    time.sleep(5)

print("")
end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())

###Code to close torrent session


In [0]:
try:
    ses.remove_torrent(handle)
    print("Session End")
except (RuntimeError, NameError) as e:
    print(e)

###Unmount Google Drive
To sync data written and unmount Goodle Drive

In [0]:
drive.flush_and_unmount()
print("Drive unmounted")